In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import geopandas as gpd
import geoplot.crs as gcrs
import geoplot as gplt
import re
from geopy.distance import great_circle
from shapely.geometry import Polygon, LineString, Point, MultiPolygon
from shapely import wkt

In [4]:
df = pd.read_csv('data/df_combined_cleaned_20250902.csv')

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_54818/2722503206.py:1: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/df_combined_cleaned_20250902.csv')


In [12]:
df2 = pd.read_csv('data/df_combined2_cleaned_20250902.csv')

In [5]:
df.head()

,timestamp,bboxid,latitude,longitude,ground_truth,category,source,geometry_bbox,center_latlon,year,month,day
0,2016-11-09 14:15:57.000,bbox_10_14,37.720555,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
1,2016-11-09 14:15:58.000,bbox_10_14,37.720457,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
2,2016-11-09 14:15:59.000,bbox_10_14,37.720360,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
3,2016-11-09 14:16:00.000,bbox_10_14,37.720258,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
4,2016-11-09 14:16:01.000,bbox_10_14,37.720156,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9


In [6]:
df['source'].value_counts()

source
Mapillary    1803853
311           328980
311            31155
Name: count, dtype: int64

In [13]:
df2['source'].value_counts()

source
Mapillary    377844
311          325796
Name: count, dtype: int64

In [10]:
# Convert data type of 'source' column to string
df['source'] = df['source'].astype(str)
df[df['source'] == '311'].shape

(360135, 12)

In [11]:
df['source'].value_counts()

source
Mapillary    1803853
311           360135
Name: count, dtype: int64

In [14]:
df.head()

,timestamp,bboxid,latitude,longitude,ground_truth,category,source,geometry_bbox,center_latlon,year,month,day
0,2016-11-09 14:15:57.000,bbox_10_14,37.720555,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
1,2016-11-09 14:15:58.000,bbox_10_14,37.720457,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
2,2016-11-09 14:15:59.000,bbox_10_14,37.720360,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
3,2016-11-09 14:16:00.000,bbox_10_14,37.720258,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9
4,2016-11-09 14:16:01.000,bbox_10_14,37.720156,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9


In [15]:
# Convert df to geo data frame
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
df2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude, df2.latitude), crs=4326)

In [17]:
import folium

In [18]:
df.head()

,timestamp,bboxid,latitude,longitude,ground_truth,category,source,geometry_bbox,center_latlon,year,month,day,geometry
0,2016-11-09 14:15:57.000,bbox_10_14,37.720555,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9,POINT (-122.48486 37.72055)
1,2016-11-09 14:15:58.000,bbox_10_14,37.720457,-122.484861,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9,POINT (-122.48486 37.72046)
2,2016-11-09 14:15:59.000,bbox_10_14,37.720360,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9,POINT (-122.48486 37.72036)
3,2016-11-09 14:16:00.000,bbox_10_14,37.720258,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9,POINT (-122.48486 37.72026)
4,2016-11-09 14:16:01.000,bbox_10_14,37.720156,-122.484856,0.0,Homeless Tents,Mapillary,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",2016,11,9,POINT (-122.48486 37.72016)


In [19]:
# Create a map centered at San Francisco
m = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add points to the map
for idx, row in df.iterrows():
    folium.Circle(
        location = [row['latitude'], row['longitude']],
        popup = row['source'],
    ).add_to(m)

In [ ]:
m

In [ ]:
from tqdm import tqdm

# Create a 311 map centered at San Francisco
m2 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add points to the map
for _, row in tqdm(df[(df['source']=='311') & (df['ground_truth']==1)].iterrows(), 
total=df[(df['source']=='311') & (df['ground_truth']==1)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .5,
        color = "#e14b31",
        fill = True,
        opacity = 0.3,
    ).add_to(m2)

for _, row in tqdm(df[(df['source']=='311') & (df['ground_truth']==0)].iterrows(), 
total=df[(df['source']=='311') & (df['ground_truth']==0)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = 1,
        color = "#1984c5",
        fill = True,
        opacity = 0.7,
    ).add_to(m2)



In [ ]:
m2

In [ ]:
m2.save('data/311_ground_truth_map_20250902.jpg')